In [ ]:
options(pkgType = "binary", repos = "https://cran-archive.r-project.org/")

In [ ]:
#install.packages("tidyverse")
#install.packages("rvest")
#install.packages("stringr")
#install.packages("shiny")
#install.packages("pdftools")
library(tidyverse)
library(rvest)
library(stringr)
library(shiny)
library(pdftools)

In [ ]:
#Extraer tipo de cambio
base_url <- "https://www.sbs.gob.pe/app/pp/sistip_portal/paginas/publicacion/tipocambiopromedio.aspx"
webpage <- read_html(base_url)
tipo_de_cambio <- html_nodes(webpage, "td.APLI_fila2:last-child")
tipo_de_cambio <- as.character(html_text(tipo_de_cambio))
tipo_de_cambio <- as.numeric(tipo_de_cambio[1])

#Creación de tabla ficticia
datos=data.frame(
  empresas=round(runif(60, min = 1, max = 5),0),
  fecha=seq(as.Date("2018-01-01"), as.Date("2022-12-31"), by = "month"),
  costo_total=runif(60, min = 1000, max = 5000)
)
datos=datos %>%
  mutate(costo_total_soles=costo_total*tipo_de_cambio, mes=month(fecha),año=year(fecha))
datos$empresas[datos$empresas==1]="Empresa 1"
datos$empresas[datos$empresas==2]="Empresa 2"
datos$empresas[datos$empresas==3]="Empresa 3"
datos$empresas[datos$empresas==4]="Empresa 4"
datos$empresas[datos$empresas==5]="Empresa 5"

#Automático
ruta_pdf="C:/Users/user/Desktop/Ciclo 7/Info nego/Trabajo/Prueba.pdf" #Poner ruta del pdf / y agregarle al final el nombre del pdf
file=pdf_text(ruta_pdf)
cat(file)
palabras <- str_extract_all(file, "\\w+")[[1]]
#CÓDIGO
#DATOS DEL PROVEEDOR
proveedor=palabras[which(palabras=="Empresa")+1]
proveedor=paste(substr(proveedor,1,7),substr(proveedor,8,8),sep = " ")
#CREAR LA TABLA CON LOS VALORES
tipo_de_cambio_dolares=as.numeric(paste(palabras[which(palabras=="cambio")+1],palabras[which(palabras=="cambio")+2],sep = "."))
valor_fob=as.numeric(palabras[which(palabras=="fob")+1])
flete=as.numeric(palabras[which(palabras=="Flete")+1])
seguro=as.numeric(palabras[which(palabras=="Seguro")+1])
valor_cif=valor_fob+flete+seguro
cuadro=data.frame(proveedor, tipo_de_cambio_dolares,valor_fob,flete,seguro,valor_cif)

#cuadro=mutate(cuadro, valor_fob_soles=tipo_de_cambio_dolares*valor_fob)
#cuadro=mutate(cuadro,flete_soles=tipo_de_cambio_dolares*flete)
#cuadro=mutate(cuadro,seguro_soles=tipo_de_cambio_dolares*seguro)
#cuadro=mutate(cuadro,valor_cif_soles=tipo_de_cambio_dolares*valor_cif)

#OTROS COSTOS
#IMPUESTOS
IGV=valor_cif*(16/100)
IPM=valor_cif*(2/100)
Percepción=palabras[which(palabras=="Percepción")+1]
if(Percepción=="Sí"){
  Percepción=(3.5/100)*valor_cif
}else{
  Percepción=0
}
total_impuestos=IGV+IPM+Percepción
#ALMACEN
Costo_transporte_almacenje_gastoslocales=as.numeric(palabras[which(palabras=="Gastos")+1])
#MULTAS - COMISIONES
comisión_interbancaria=as.numeric(palabras[which(palabras=="interbancaria")+1])
multas=palabras[which(palabras=="Multa")+1]
if(multas=="Sí"){
  multas=multas=palabras[which(palabras=="pagar")+1]
}else{
  multas=0
}
#TOTAL OTROS COSTOS
otros_costos=total_impuestos+comisión_interbancaria+multas+Costo_transporte_almacenje_gastoslocales
cuadro=cbind(cuadro,otros_costos)

#COSTO TOTAL
costo_total=round(valor_cif+otros_costos,2)
costo_total_soles=round(costo_total*tipo_de_cambio_dolares,2)
cuadro=cbind(cuadro,costo_total)
cuadro=cbind(cuadro,costo_total_soles)

print(cuadro)

#Parcialmente automático
#Interfaz
ui <- fluidPage(
  textInput("name", "Ingresa el nombre de la empresa"),
  dateInput("fecha", "Ingresa la fecha del registro",value = Sys.Date()),
  actionButton("button1", "Ingresar"),
  textOutput("output"),
  titlePanel("Costos principales"),
  textInput("valor_fob", "Ingrese el valor FOB (en dólares)"),
  textInput("seguro", "Ingrese el precio del seguro (en dólares)"),
  textInput("flete", "Ingrese el monto del flete (en dólares)"),
  selectInput("tipodecambio", "Seleccionar tipo de cambio",
              choices = c("Hoy","Manual"),
              selected = "Hoy"),
  numericInput("tcd", "Tipo de cambio",tipo_de_cambio),
  textOutput("valor_cif"),
  titlePanel("Impuestos"),
  selectInput("percepción", "¿Es necesario percepción?",
              choices = c("Sí","No"),
              selected = "No"),
  textOutput("imp"),
  titlePanel("Otros costos"),
  textInput("gastos1", "Ingrese la suma de gastos locales, almacenaje y transporte (en dólares)"),
  textInput("comisión", "Ingresar monto de la comisión interbancaria"),
  selectInput("multa", "¿Incurrió en multa?",
              choices = c("Sí","No"),
              selected= "No"),
  numericInput("monto_multa", "Ingresar monto de la multa",0),
  textOutput("oc"),
  textOutput("fin"),
  titlePanel("Gráficos"),
  sidebarLayout(
    sidebarPanel(
      selectInput("medida_tiempo", "Seleccionar escala",
                  choices = c("Año","Mes"),
                  selected = "Año"),
      selectInput("moneda", "Seleccionar la moneda a usar",
                  choices = c("Dolar","Sol"),
                  selected = "Dolar")
    ),
    mainPanel(
      tabsetPanel(
        tabPanel("Gráfico de Barras", plotOutput("grafico_barras")),
        tabPanel("Tabla de Datos", dataTableOutput("tabla_datos")),
      )
    )
  )
)

#Servidor
server <- function(input, output){
  #Eventos observados
  observeEvent(input$button1,{
    output$output <- renderText({
      paste("Está trabajando con",input$name, "el", input$fecha)
    })
  })
  #Eventos reactivos
  tcd=reactive({
    if(input$tipodecambio=="Hoy"){
      resultado_tcd=tipo_de_cambio
    }else{
      resultado_tcd=input$tcd
    }
    resultado_tcd
  })
  vcif=reactive({
    as.numeric(input$valor_fob) + as.numeric(input$seguro) + as.numeric(input$flete)
  })
  impuesto=reactive({
    IGV=vcif()*(16/100)
    IPM=vcif()*(2/100)
    if(input$percepción=="Sí"){
      per=(vcif()+IGV+IPM)*(3.5/100)
      resultado_imp=IGV+IPM+per
    }else{
      resultado_imp=IGV+IPM
    }
    resultado_imp
  })
  otc=reactive({
    if(input$multa=="Sí"){
      resultado_otc=as.numeric(input$gastos1)+as.numeric(input$comisión)+input$monto_multa
    }else{
      resultado_otc=as.numeric(input$gastos1)+as.numeric(input$comisión)
    }
    resultado_otc
  })
  costo_total=reactive({
    vcif()+impuesto()+otc()
  })
  costo_total_soles=reactive({
    costo_total()*tcd()
  })
  #Impresión de datos
  output$valor_cif <- renderPrint({
    paste("Valor CIF:",vcif())
  })
  output$imp <- renderPrint({
    paste("Valor de impuestos:",impuesto())
  })
  output$oc <- renderPrint({
    paste("Otros costos:",otc())
  })
  output$fin <- renderPrint({
    paste("Costo total:",costo_total(),"y costo total en soles:", costo_total_soles())
  })
  #Registro - se intentó
  #empresa=reactive({
  #  input$name
  #})
  #fecha=reactive({
  #  input$fecha
  #})
  #datos1=reactive({
  #  registro=c(empresa(),fecha(),costo_total(),costo_total_soles(),month(fecha()),year(fecha()))
  #  rbind(datos,registro)
  #})
  #Gráfico de barras
  #Filtro:
  datos_filtrados=reactive({
    if(input$medida_tiempo=="Año"){
      if(input$moneda=="Dolar"){
        datos_filtrados=datos%>%
          select(costo_total,año)%>%
          group_by(año)%>%
          mutate(costo_total1=sum(costo_total))%>%
          select(costo_total1,año)%>%
          unique()%>%
          rename(fecha=año)%>%
          arrange(fecha)
      }else{
        datos_filtrados=datos%>%
          select(costo_total_soles,año)%>%
          group_by(año)%>%
          mutate(costo_total1=sum(costo_total_soles))%>%
          select(costo_total1,año)%>%
          unique()%>%
          rename(fecha=año)%>%
          arrange(fecha)
      }
    }else{
      if(input$moneda=="Dolar"){
        datos_filtrados=datos%>%
          select(costo_total,mes)%>%
          group_by(mes)%>%
          mutate(costo_total1=sum(costo_total))%>%
          select(costo_total1,mes)%>%
          unique()%>%
          rename(fecha=mes)%>%
          arrange(fecha)
      }else{
        datos_filtrados=datos%>%
          select(costo_total_soles,mes)%>%
          group_by(mes)%>%
          mutate(costo_total1=sum(costo_total_soles))%>%
          select(costo_total1,mes)%>%
          unique()%>%
          rename(fecha=mes)%>%
          arrange(fecha)
      }
    }
    datos_filtrados
  })

  #Gráfico
  output$grafico_barras <- renderPlot({
    datos_filtrados <- datos_filtrados()
    
    ggplot(datos_filtrados, aes(x = fecha, y = costo_total1)) +
      geom_bar(stat = "identity") + 
      labs(x = "Fecha", y = "Costo total", title = "costo (en dólares)")
  })
  # Tabla de datos:
  output$tabla_datos <- renderDataTable({
    datos_filtrados()
  })
}

shinyApp(ui, server)